In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import fftpack
from scipy.fftpack import dct, idct
from skimage import data

plt.gray()
%matplotlib inline

In [15]:
def dct2(a):
    return dct(dct(a.T).T)
def idct2(a):
    return idct(idct(a.T).T)
def gnoise(a, sig):
    return np.clip(a+sig*np.random.randn(*a.shape), a.min(), a.max())
def spnoise(a, p):
    n = np.random.rand(*a.shape)
    l = n<p/2
    u = n>(1-p/2)
    return a * ~(l|u) + u*a.max()

In [13]:
im = data.camera()